# Hands on with pandas

This notebook was written by Tim Hillel (tim.hillel@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557).

Please contact before distributing or reusing the material below.

## Overview

This notebook will introduce the case-study dataset for the course, using the pandas python library.

## Pandas

Pandas is a library which replicates the dataframe functionality of R in python. 

It is a wrapper around a numpy array with a *header* for each *column* and a unique *index* for each *row*.
(The difference between a pandas `DataFrame` and a numpy `Array` is similar to the difference between a `dictionary`  and a `list`.)

If you have not used R before, you can consider pandas to be the *excel* of Python!

Let's import pandas. As with numpy, we give pandas an alias, this time `pd`

In [ ]:
import pandas as pd

### Importing data

You can manually enter data in a pandas dataframe in Python, as we did with numpy in the last notebook.

However, pandas is much more useful when we import data from elsewhere! Pandas supports many file types and interfaces, including *SQL, Excel,* and *JSON*. Today however, we will be importing a *csv* file.

The data file is in the data directory. We can open it using a relative file path.

In [ ]:
df = pd.read_csv('data/dataset.csv')

The dataframe has many attributes. Have a look at the shape of the dataframe, and the names of the columns. 

In [ ]:
df.shape


In [ ]:
df.columns


We can use the method `head` to look at the first 5 rows.

In [ ]:
df.head()

### Pandas series & boolean masks

Every column in the dataframe is a series. We can access the series using the column name. Remember, we use square brackets for indexing!

In [ ]:
dist = df['distance']

A pandas series is a numpy vector with an index and a name. We can use many similar methods and attributes to numpy arrays. 

Try using the relevant methods to output the mean, min, and max distance. Use a format string to output them in a useful sentence.

In [ ]:
print(f'''The mean distance is {dist.mean()}, 
      the minimum distance is {dist.min()}, 
      and the max distance is {dist.max()}''')


We can also use boolean conditions on a series, e.g. dist<1000. 

Use this to select all trips in the dataset greater than 5km (5000m), and store it as a new dataframe `long_trips`

In [ ]:
long_trips = df[dist>5000]


How many trips (rows) are in long_trips?

In [ ]:
long_trips.shape


As well as using the column name to index each column, we can also access each column as an `attribute` of the dataframe. This is particularly useful for using tab autocomplete!

In [ ]:
long_trips.travel_mode.head()

Try to output the unique values in `long_trips.travel_mode`

In [ ]:
long_trips.travel_mode.unique()


We could also count the values

In [ ]:
long_trips.travel_mode.value_counts()

Try comparing the ratios of the different modes for the long_trips and trips shorter than 1000m. *Hint* divide the value counts by the number of trips

In [ ]:
print(long_trips.travel_mode.value_counts()/long_trips.shape[0])
print(df[df.distance<1000].travel_mode.value_counts()/df[df.distance<1000].shape[0])


#### Selecting multiple columns

We have seen that we can access a single `series` (column) by indexing the name of the column, and we could also select rows by using a boolean mask. 

What if we want to select a subset of the columns? 

We can use a list of the column names we want to select!


In [ ]:
smalldf = df[['travel_mode', 'distance']]
smalldf.head()

Using a comprehension with the `travel_modes` iterator created below, create a dictionary where the `key` is each transport mode, and the `value` is the mean distance for that mode (from `smalldf`)

In [ ]:
travel_modes = smalldf.travel_mode.unique()
# use dictionary comprehension with travel_modes and smalldf
{m: smalldf[smalldf.travel_mode==m].distance.mean() for m in travel_modes}


Lets add a few more columns. Use a list comprehension to create a list of all columns names which contain the string `dur`. 
Append the values `travel_mode` and `distance` to this list, and use it to select these columns from `df` to form a new dataframe `dur_df`.

*Hints*: Remember `df.columns` returns a list of the column names, and we can check if something (e.g. a substring or integer) is in something else (e.g. a string or list) using the `in` keyword!

In [ ]:
dur_df = df[['travel_mode', 'distance'] + [c for c in df.columns if 'dur' in c]]
dur_df.head()


Pandas dataframes have a useful method, `describe`, which provide basic statistics about all of the numerical columns in a dataframe

In [ ]:
dur_df.describe()

## Exercise

Using the `dur_df` dataframe, and the matplotlib plotting interface produce two plots:

1. The first plot should be a scatter plot of *distance* vs *duration* for the four transport modes (`['dur_walking', 'dur_cycling', 'dur_pt_total', 'dur_driving']`). 
Use a different colour for each mode, with a marker of a point ('.') and a size of 1. 
Remember to add a legend and label your axis. 
Save it as a pdf file, with a name of your choice.

2. The second plot should be a *kernel density* plot of the distance distribution for each mode. Use the density function from the direct pandas plotting interface (e.g. `df.plot.density()`)
Again, label the axis and add a legend.
Save it as a png file, with a name of your choice

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
modes = ['walking', 'cycling', 'pt_total', 'driving']
colors = ['blue', 'orange', 'magenta', 'green']
for i, mode in enumerate(modes):
    dur_df.plot.scatter(x='distance', y=f'dur_{mode}', 
                        s=1, ax=ax, marker='.', label = mode, color = colors[i])
ax.set_ylabel('duration')
fig.savefig('tim_scatter.pdf')


In [ ]:
fig, axs = plt.subplots(4, sharex=True, figsize=(8,12))
modes = ['walk', 'cycle', 'pt', 'drive']
colors = ['blue', 'orange', 'magenta', 'green']
for i, mode in enumerate(modes):
    df[df.travel_mode==mode].distance.plot.density(ax=axs[i], color=colors[i], label=mode)
    axs[i].legend()
    axs[i].set_ylim(0,.001)
    axs[i].set_xlim(-500,5000)
axs[3].set_xlabel('Duration')
fig.savefig('tim_density.png')
